In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

configs = [
    dict(
        symbol='NDX', timeframe='H1', direction='buy', pruner_type='successive',
        train_start=datetime(2018,2,1), train_end=datetime(2025,2,1),
        test_start=datetime(2023,2,1),  test_end=datetime(2024,2,1),
        label_method='random', search_type='clusters', search_subtype='lgmm',
        n_models=1, n_jobs=8, debug=False,
    ),
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='successive',
        train_start=datetime(2018,2,1), train_end=datetime(2025,2,1),
        test_start=datetime(2023,2,1),  test_end=datetime(2024,2,1),
        label_method='random', search_type='clusters', search_subtype='lgmm',
        n_models=1, n_jobs=8, debug=False,
    ),
    dict(
        symbol='GDAXI', timeframe='H1', direction='buy', pruner_type='successive',
        train_start=datetime(2018,2,1), train_end=datetime(2025,2,1),
        test_start=datetime(2023,2,1),  test_end=datetime(2024,2,1),
        label_method='random', search_type='clusters', search_subtype='lgmm',
        n_models=1, n_jobs=8, debug=False,
    ),
]

# Crear tag para cada configuración
for cfg in configs:
    # Construir el tag asegurando que no haya dobles guiones bajos por campos vacíos
    tag_parts = [
        cfg['symbol'],
        cfg['timeframe'],
        cfg['direction'],
        cfg['label_method'][:2],
        cfg['search_type'][:3],
        (cfg.get('search_subtype') or '')[:2],
    ]
    # Filtrar partes vacías y unir con "_"
    cfg["tag"] = "_".join([part for part in tag_parts if part]).strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None

n_trials = 5000
if study:
    n_trials = len(study.trials) + n_trials

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD_H1_buy_ra_clu_lg] modelo 0 trial 1/5000 score=-1.000000 avg=76.42s mem=523.97MB 
[NDX_H1_buy_ra_clu_lg] modelo 0 trial 11/5000 score=-1.000000 avg=7.51s mem=1033.27MB 
[XAUUSD_H1_buy_ra_clu_lg] modelo 0 trial 2/5000 score=-1.000000 avg=70.24s mem=527.13MB 
[NDX_H1_buy_ra_clu_lg] modelo 0 trial 12/5000 score=-1.000000 avg=15.95s mem=1288.89MB 
[XAUUSD_H1_buy_ra_clu_lg] modelo 0 trial 3/5000 score=-1.000000 avg=75.21s mem=1047.93MB 
[NDX_H1_buy_ra_clu_lg] modelo 0 trial 13/5000 score=-1.000000 avg=17.58s mem=1284.56MB 
[GDAXI_H1_buy_ra_clu_lg] modelo 0 trial 1/5000 score=-1.000000 avg=243.96s mem=1351.26MB 
[XAUUSD_H1_buy_ra_clu_lg] modelo 0 trial 4/5000 score=-1.000000 avg=63.38s mem=1059.66MB 
[NDX_H1_buy_ra_clu_lg] modelo 0 trial 14/5000 score=-1.000000 avg=18.52s mem=1271.87MB 
[XAUUSD_H1_buy_ra_clu_lg] modelo 0 trial 5/5000 score=-1.000000 avg=54.27s mem=1060.39MB 
[GDAXI_H1_buy_ra_clu_lg] modelo 0 trial 2/5000 score=-1.000000 avg=139.68s mem=1342.54MB 
[XAUUSD_H1_buy_ra_clu